<a href="https://colab.research.google.com/github/Steveglia/PneumoNet/blob/main/Chest_X_Ray_Medical_Images_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PneumoNet: Empowering Healthcare with AI-driven Pneumonia Detection #

**<font size="3">Early and accurate diagnosis in modern healthcare plays a pivotal role in improving patient outcomes. Our project, "PneumoNet," harnesses the power of artificial intelligence to revolutionize the detection of pneumonia from chest X-rays. Pneumonia, a prevalent respiratory condition, requires prompt identification for timely intervention and optimal patient care.
Our project utilizes a diverse dataset comprising chest X-ray images from reputable healthcare repositories. The dataset encompasses normal and pneumonia-affected cases, ensuring a robust model capable of discerning subtle radiological patterns indicative of the disease. Leveraging advanced deep learning techniques, we aim to develop an intelligent diagnostic tool capable of swiftly and accurately identifying pneumonia in chest X-ray images.
The significance of our project lies in its potential to expedite diagnosis, reduce the workload on healthcare professionals, and enhance overall healthcare efficiency. With the integration of our AI-driven diagnostic solution, we aspire to facilitate timely treatment initiation, thereby improving patient outcomes and alleviating the strain on healthcare resources. PneumoNet represents a leap towards enhancing diagnostic capabilities and ultimately contributing to advancing healthcare practices.</font>**

##Importing necessary libraries##


In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import pathlib
import imageio
%matplotlib inline

##Oversampling - Blancing the dataset##

**<font size="3">Generating augmented images from the normal class to balance the training dataset. This ensures diversity in the augmented dataset by applying different data augmentation to each halves of the minority class. This helps prevent the model from overfitting to a specific set of augmented images and improves its ability to generalize to new, unseen data.**</font>

In [ ]:
# Generate augmented images
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

# Specify the directory name and its location
desired_directory = "/ChestXRay2017/chest_xray/train/NORMAL"

# Check if the directory already exists
if not os.path.exists(desired_directory):
    # If it doesn't exist, create the directory
    os.makedirs(desired_directory)
    print(f"Directory '{desired_directory}' created successfully.")
else:
    print(f"Directory '{desired_directory}' already exists.")

# Example: Specify the directory containing your images
image_directory = "/ChestXRay2017/chest_xray/train/NORMAL/"

# Get a list of all image file names in the directory
image_files = [f for f in os.listdir(image_directory) if f.endswith(('.jpeg'))]

# Load the images using OpenCV
images = [cv2.imread(os.path.join(image_directory, f)) for f in image_files]

# Convert the images to NumPy arrays
image_arrays = np.array(images)

# Define your data augmentation parameters for the first half
datagen_first_half = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.15,
    zoom_range=0.15,
    fill_mode='nearest',
    horizontal_flip=False
)

# Define your data augmentation parameters for the second half
datagen_second_half = ImageDataGenerator(
    rotation_range=13,  # Adjusted rotation
    width_shift_range=0.1,  # Adjusted width shift
    height_shift_range=0.1,  # Adjusted height shift
    shear_range=0.25,  # Adjusted shear
    zoom_range=0.25,  # Adjusted zoom
    fill_mode='nearest',
    horizontal_flip=False
)

# Specify the desired total number of augmented images
desired_total_images = 2062
batch_size = 16
epochs = desired_total_images // batch_size
first_half_count = 0
second_half_count = 0

# Assuming num_batches_per_epoch is calculated based on the original dataset size
num_batches_per_epoch = len(image_arrays) // batch_size

# Calculate the number of images to generate with slightly different data augmentation
num_slightly_different = batch_size // 2

# Counter to keep track of the current original image index
generated_images_count = 0

for epoch in range(epochs):
    for batch in range(num_batches_per_epoch):

        # Generate and save augmented normal images
        for i in range(batch_size):
            # Break the loop if the desired number of augmented images is reached
            if generated_images_count >= desired_total_images:
                break

            # Choose different data augmentation for at least half of the images
            if i < num_slightly_different:
                augmented_image = datagen_first_half.random_transform(image_arrays[i])
                first_half_count += 1
            else:
                augmented_image = datagen_second_half.random_transform(image_arrays[i - num_slightly_different])
                second_half_count += 1

            # Update the count for each augmented image
            generated_images_count += 1

            # Save each augmented image with a unique filename
            save_path = os.path.join("/ChestXRay2017/chest_xray/train/NORMAL", f"IM-{generated_images_count}.jpeg")
            plt.imsave(save_path, augmented_image)
print(f"{first_half_count} images created with the first data augmentation")
print(f"{second_half_count} images created with the second data augmentation")
print(f"{second_half_count + first_half_count} images created")

##Verify all normal images are unique##

In [ ]:
# Check the number of unique images in a directory
import os
from PIL import Image

def count_unique_images(directory_path):
    # Initialize a set to store unique image hashes
    unique_image_hashes = set()

    # Iterate through files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if the file is an image
        if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            try:
                # Open the image and calculate its hash
                with Image.open(file_path) as img:
                    image_hash = hash(img.tobytes())
                    unique_image_hashes.add(image_hash)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Return the count of unique images
    return len(unique_image_hashes)

# Replace 'your_directory_path' with the path to your image directory
directory_path = '/ChestXRay2017/chest_xray/train/NORMAL'
unique_image_count = count_unique_images(directory_path)

print(f"Number of unique images in {directory_path}: {unique_image_count}")

##Dataset File Paths and Summary Setup for Chest X-ray Analysis##

**<font size="3">Set up file paths for a chest X-ray dataset stored in different directories for training, testing, and validation. Then creates lists of file paths for images in categories like pneumonia and normal for each dataset split (train, test, and val). Finally, it prints the total count of images in each category to provide a quick summary of the dataset.**</font>

In [ ]:
# Define the base directory for the chest X-ray dataset
base_dir = '/ChestXRay2017/chest_xray/'

# Define directories for training data
train_pneumonia_dir = base_dir + 'train/PNEUMONIA/'
train_normal_dir = base_dir + 'train/NORMAL/'

# Define directories for testing data
test_pneumonia_dir = base_dir + 'test/PNEUMONIA/'
test_normal_dir = base_dir + 'test/NORMAL/'

# Define directories for validation data
val_normal_dir = base_dir + 'val/NORMAL/'
val_pneumonia_dir = base_dir + 'val/PNEUMONIA/'

# Create lists of file paths for images in each category
train_pn = [train_pneumonia_dir + "{}".format(i) for i in os.listdir(train_pneumonia_dir)]
train_normal = [train_normal_dir + "{}".format(i) for i in os.listdir(train_normal_dir)]

test_normal = [test_normal_dir + "{}".format(i) for i in os.listdir(test_normal_dir)]
test_pn = [test_pneumonia_dir + "{}".format(i) for i in os.listdir(test_pneumonia_dir)]

val_pn = [val_pneumonia_dir + "{}".format(i) for i in os.listdir(val_pneumonia_dir)]
val_normal = [val_normal_dir + "{}".format(i) for i in os.listdir(val_normal_dir)]

# Print total counts of images in different categories
print("Total images:", len(train_pn + train_normal + test_normal + test_pn + val_pn + val_normal))
print("Total pneumonia images:", len(train_pn + test_pn + val_pn))
print("Total Normal images:", len(train_normal + test_normal + val_normal))

Total images: 5873
Total pneumonia images: 4282
Total Normal images: 1591


## Dataset Preprocessing & Visualization - Original Dataset##
**<font size="3">The dataset is relatively small, consisting of 5873 chest X-ray images with pneumonia and normal conditions. To maximize the utility of the limited data, we adopt an 80-15-5 split strategy, dividing the dataset into training, testing, and validation sets. This distribution ensures that a substantial portion (80%) is allocated to training, allowing the model to learn patterns effectively. The 15% reserved for testing is an independent benchmark to evaluate the model's generalization performance. The remaining 5% set aside for validation aids in fine-tuning the model and preventing overfitting. This partitioning strategy balances training with available data and rigorously assesses the model's performance on unseen instances, considering the constraints posed by the dataset's size. </font>**

In [ ]:
# Dataset Splitting (train 80%, test 15%, and validation 5%)

# Combining pneumonia and normal chest X-ray images into two Python lists
pn = train_pn + test_pn + val_pn
normal = train_normal + test_normal + val_normal

# Splitting the dataset into train set, test set, and validation set
train_imgs = pn[:3418] + normal[:1224]  # 80% of 4642 Pneumonia and normal chest X-ray are 3418 and 1224, respectively.
test_imgs = pn[3418:4059] + normal[1224:1502] # 15% of 919 Pneumonia and normal chest X-ray are 641 and 278, respectively.
val_imgs = pn[4059:] + normal[1502:] # 5% of 312 Pneumonia and normal chest X-ray are 223 and 89, respectively.

# Displaying the distribution of images in each set
print("Total Train Images: %s containing %s pneumonia and %s normal images"
      % (len(train_imgs), len(pn[:3418]), len(normal[:1224])))
print("Total Test Images: %s containing %s pneumonia and %s normal images"
      % (len(test_imgs), len(pn[3418:4059]), len(normal[1224:1502])))
print("Total Validation Images: %s containing %s pneumonia and %s normal images"
      % (len(val_imgs), len(pn[4059:]), len(normal[1502:])))

# Randomly shuffling the images in each set
import random
random.shuffle(train_imgs)
random.shuffle(test_imgs)
random.shuffle(val_imgs)

## Dataset Preprocessing & Visualization - Augmented Dataset##
**<font size="3">The dataset is relatively small, consisting of 8529 chest X-ray images with pneumonia and normal conditions. To maximize the utility of the limited data, we adopt an 86-13-1 split strategy, dividing the dataset into training, testing, and validation sets. This distribution ensures that a substantial portion (86%) is allocated to training, allowing the model to learn patterns effectively. The 13% reserved for testing is an independent benchmark to evaluate the model's generalization performance. The remaining 1% set aside for validation aids in fine-tuning the model and preventing overfitting. This partitioning strategy balances training with available data and rigorously assesses the model's performance on unseen instances, considering the constraints posed by the dataset's size. </font>**

In [ ]:
# Dataset Splitting (train 86%, test 13%, and validation 1%)

# Combining pneumonia and normal chest X-ray images into two Python lists
pn = train_pn + test_pn + val_pn
normal = train_normal + test_normal + val_normal

# Splitting the dataset into train set, test set, and validation set
train_imgs = pn[:3411] + normal[:3411]  # 86% of 6822 Pneumonia and normal chest X-ray are 3411 and 3411, respectively.
test_imgs = pn[3411:4222] + normal[3411:3615] # 13% of 919 Pneumonia and normal chest X-ray are 811 and 204, respectively.
val_imgs = pn[4222:] + normal[3615:] # 1% of 312 Pneumonia and normal chest X-ray are 60 and 38, respectively.

# Displaying the distribution of images in each set
print("Total Train Images: %s containing %s pneumonia and %s normal images"
      % (len(train_imgs), len(pn[:3411]), len(normal[:3411])))
print("Total Test Images: %s containing %s pneumonia and %s normal images"
      % (len(test_imgs), len(pn[3411:4222]), len(normal[3411:3615])))
print("Total Validation Images: %s containing %s pneumonia and %s normal images"
      % (len(val_imgs), len(pn[4222:]), len(normal[3615:])))

# Randomly shuffling the images in each set
import random
random.shuffle(train_imgs)
random.shuffle(test_imgs)
random.shuffle(val_imgs)

## Preprocess the images ##
**<font size="3">The purpose of this code is to prepare a dataset for training a machine learning model on chest X-ray images, ensuring uniformity in size, color channels, and labeling for further analysis and model development. The preprocessing steps help create a consistent and standardized input for a machine learning algorithm. </font>**

In [ ]:
import cv2
img_size = 224

def preprocess_image(image_list):
    """
    Preprocesses a list of image file paths.

    Args:
    - image_list (list): List of file paths for images.

    Returns:
    - X (list): Processed images.
    - y (list): Labels (0 for Normal or 1 for Pneumonia).
    """

    X = [] # Images
    y = [] # Labels (0 for Normal or 1 for Pneumonia)
    count = 0

    for image in image_list:

        try:

            # Read and convert the image
            img = cv2.imread(image, cv2.IMREAD_GRAYSCALE) # Read the image in grayscale
            img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_CUBIC) # Resize the image to the target size
            img = np.dstack([img, img, img]) # Convert the grayscale image to a 3D RGB image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert the image from BGR to RGB color space. This is done to ensure the image has three channels (RGB).

            # Normalalize Image
            img = img.astype(np.float32)/255.

            count += 1
            X.append(img)

        except:
            continue

        # Get the labels
        if 'NORMAL' in image or 'IM' in image:
            y.append(0)
        elif 'virus' in image or 'bacteria' in image:
            y.append(1)

    return X, y

##Preprocess train images##

In [ ]:
# get the labels for train set
X, y = preprocess_image(train_imgs)

##Check if all train images getting labels##

In [ ]:
arr = y
# Get a tuple of unique values & their frequency in numpy array
uniqueValues, occurCount = np.unique(arr, return_counts=True)

# Informative printing labels alongside their occurrence counts.
for label, count in zip(uniqueValues, occurCount):
    print(f"Label {label}: Occurs {count} times")

##Display some images from train set##

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 5))
k = 1
for i in range(4):
    a = fig.add_subplot(1, 4, k)
    if (y[i] == 0):
        a.set_title('Normal')
    else:
        a.set_title('Pneumonia')

    plt.imshow(X[i])
    k += 1;

##Preprocess test images##

In [ ]:
# get the labels for test set
P, t = preprocess_image(test_imgs)

##Check if all test images getting labels##

In [ ]:
arr = t
# Get a tuple of unique values & their frequency in numpy array
uniqueValues, occurCount = np.unique(arr, return_counts=True)

# Informative printing labels alongside their occurrence counts.
for label, count in zip(uniqueValues, occurCount):
    print(f"Label {label}: Occurs {count} times")

##Display some images from test set##

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 5))
k = 1
for i in range(4):
    a = fig.add_subplot(1, 4, k)
    if (t[i] == 0):
        a.set_title('Normal')
    else:
        a.set_title('Pneumonia')

    plt.imshow(P[i])
    k += 1;

##Preprocess validation images##

In [ ]:
# get the labels for validation set
K, m = preprocess_image(val_imgs)

##Check if all validation images getting labels##

In [ ]:
arr = m
# Get a tuple of unique values & their frequency in numpy array
uniqueValues, occurCount = np.unique(arr, return_counts=True)

# Informative printing labels alongside their occurrence counts.
for label, count in zip(uniqueValues, occurCount):
    print(f"Label {label}: Occurs {count} times")

##Display some images from validation set##

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 5))
k = 1
for i in range(4):
    a = fig.add_subplot(1, 4, k)
    if (m[i] == 0):
        a.set_title('Normal')
    else:
        a.set_title('Pneumonia')

    plt.imshow(K[i])
    k += 1;

##Understanding the balance or imbalance of data in different datasets##
**<font size="3">Creates a DataFrame from three arrays and visualizes the distribution of data across the 'Train', 'Test', and 'Val' datasets using countplots.</font>**

In [ ]:
import seaborn as sns

# Create DataFrame
df = pd.DataFrame()
df['Train'] = y
df['Test'] = pd.Series(t)
df['Val'] = pd.Series(m)

data_size = 3500

# Create subplots
fig, ax = plt.subplots(1, 3, figsize=(15, 7))

# Adjust countplot parameters and set y-axis limits
sns.countplot(x='Train', data=df, ax=ax[0])
ax[0].set_ylim(0, data_size)

sns.countplot(x='Test', data=df, ax=ax[1])
ax[1].set_ylim(0, data_size)

sns.countplot(x='Val', data=df, ax=ax[2])
ax[2].set_ylim(0, data_size)

# Display the figure
plt.show()

## Dealing with class imbalance with class weights - Use only for the original dataset##

**<font size="3">In imbalanced datasets, where certain classes have significantly fewer samples than others, models might be biased towards the majority class during training. Assigning class weights helps the model give more importance to the minority class, ensuring that the model is not dominated by the majority class. By using class weights during training, the model is encouraged to pay more attention to the underrepresented classes, leading to a more balanced and fair learning process.</font>**

In [ ]:
from sklearn.utils import class_weight


class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(y), # here, y contains train set label
                                                 y=y)
class_weights = dict(enumerate(class_weights))
print(class_weights)

## Prepares and processes image data ##

**<font size="3">Prepare and processes image data for a machine learning model by combining two sets of images and converting the images and labels into NumPy arrays.</font>**

In [ ]:
# Convert the processed images and labels into NumPy arrays
X_train = np.array(X)
y_train = np.array(y)
X_test = np.array(P)
y_test = np.array(t)
X_val = np.array(K)
y_val = np.array(m)

# Print the shapes of the arrays
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

##Training##

**<font size="3">We will use a batch size of 16. Batch size should be a power of 2. The batch size 16 means the model will train 16 training samples and then update its parameters once. Batch training is faster and memory efficient.</font>**

In [ ]:
# Get the length of the train, test and validation data
ntrain = len(X_train)
nval = len(X_val)
ntest = len(X_test)

# Setting batch size
batch_size = 32

## Data Augmentation ##
**<font size="3">We employ data augmentation to artificially expand our dataset, especially given its relatively small size. Mild data augmentation is applied exclusively to the training data, mirroring the real-world scenario of standardized chest X-ray images, which the machine will encounter during testing. This approach not only aids in preventing overfitting but also aligns the training process with the expected characteristics of the test data.</font>**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Data augmentation for the training set
train_datagen = ImageDataGenerator(  rotation_range=7,
                                     width_shift_range=0.05,
                                     height_shift_range=0.05,
                                     #shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True)

# Data augmentation for the test set
test_datagen = ImageDataGenerator()

# Data augmentation for the validation set
val_datagen = ImageDataGenerator()

In [ ]:
# Create the image generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
# Set image Size
img_size = 224

## Train full train set with CNN Giovanna  ##

**<font size="3"> We build a convolutional neural network (CNN) architecture from scratch. </font>**

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks1 = [
    EarlyStopping(monitor = 'loss', patience = 6),
    ReduceLROnPlateau(monitor = 'loss', patience = 3, verbose=1),
    ]

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Activation


from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense, Activation
def Giovanna():
  layers = [
        Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Flatten(),
        Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),

        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        Dense(1, activation='sigmoid')

      ]



  model = Sequential(layers)
  return model



KeyboardInterrupt: ignored

In [ ]:
optimizer = Adam(learning_rate=0.001)  # Reducing learning rate
model = Giovanna()
model.compile(optimizer = optimizer, loss = 'binary_crossentropy' , metrics = ['accuracy','mae'])


In [ ]:
# We train for 32 epochs
history = model.fit(train_generator,
                              steps_per_epoch= ntrain // 32,
                              epochs=60,
                              callbacks = callbacks1,
                              validation_data=val_generator,
                              validation_steps=nval // batch_size,
                              class_weight =class_weights,
)

## Evaluate the model on the testing data  ##

In [ ]:
# Evaluate the model on the test data
test_results = model.evaluate(test_generator, steps=ntest // batch_size, verbose=1)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

## Visualize the training and validation accuracy and loss ##


In [ ]:
# Lets plot the train and val curve
# Get the details form the history object
acc = history.history['accuracy']
val_acc = history.history['accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

## Estimation of classification performance / Result ##
**<font size="3">We will assess the classification performance of our model using various evaluation metrics. Employing multiple metrics is crucial to thoroughly evaluate the model's correctness and optimization. Our examination will include metrics such as Accuracy, Recall, Precision, F1 score, and AUC score to comprehensively gauge the model's performance. </font>**

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(X_test)

acc = accuracy_score(y_test, np.round(preds)) * 100
cm = confusion_matrix(y_test, np.round(preds))

tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

print('\n============TEST METRICS=============')
precision = tp/(tp+fp) * 100
recall = tp/(tp+fn) * 100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((history.history['accuracy'][-1]) * 100, 2)))

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True, fmt="d",)

## Visualize the ROC ##
**<font size="3">The ROC (receiver operating characteristic) curve indicates the diagnostic accuracy and porformance of a model.We show the ROC curve and also calculate AUC score</font>**

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import auc

fpr , tpr , thresholds = roc_curve ( y_test , preds)
auc_keras = auc(fpr, tpr)
print("AUC Score:",auc_keras)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % auc_keras)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## Cross-Validation##

**<font size="3">Cross-validation is a valuable technique for obtaining a robust evaluation of a model's performance, especially when dealing with limited training data. It provides insights into how well the model generalizes to different subsets, helping to make more informed decisions about its suitability for unseen data.</font>**

In [ ]:
import seaborn as sns
sns.set(style= "darkgrid", color_codes = True)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from keras.preprocessing.image import img_to_array, load_img
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

datagen = ImageDataGenerator(rotation_range=7,
                             width_shift_range=0.05,
                             height_shift_range=0.05,
                             #shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True
)


# Initialize an empty list to store accuracy scores
accuracy_scores = []
c = 1
batch = 32
# Iterate through each fold
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_generator = datagen.flow(X_train, y_train, batch_size = batch)

     # Initialize and compile the model
    model = Giovanna()
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


    print(f"starting subsequence {c}:")
    # Train the model
    model.fit(      train_generator,
                    steps_per_epoch=len(X_train) // batch,
                    epochs=60,
                    callbacks = callbacks1,
    )
    print(f"finish subsequence {c}")

    # Evaluate the model on the test set

    evaluation_results1 = model.evaluate(X_test, y_test, verbose = 0)
    print(f"Test loss: {evaluation_results1[0] * 100:.2f}%, Test accuracy: {evaluation_results1[1] * 100:.2f}%")

    y_pred_prob = model.predict(X_test)

    # Convert probabilities to binary predictions using a threshold (e.g., 0.5)
    y_pred = (y_pred_prob > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    print(f"\nThe accuracy for the subsequence {c} is: {accuracy}")
    model.save(f'mk{c}.h5')
    c += 1
    tf.keras.backend.clear_session()

# Print the cross-validated accuracy

print("Cross-validated Accuracy: {:.2f}%".format(np.mean(accuracy_scores) * 100))
print(f"\n this is the accuracy for:\nSubsequence 1 {accuracy_score[0]}\nSubsequence 2 {accuracy_score[1]}\nSubsequence 3 {accuracy_score[2]}\nSubsequence 4 {accuracy_score[3]}\nSubsequence 5 {accuracy_score[4]} ")


## Multi model combination ##

**<font size="3">After developing four distinct models, we merge their outcomes to enhance accuracy.
The algorithm assesses predictions from each model, returning the most probable result.
However, when all four models yield identical predictions for opposing outcomes, we prioritize the best prediction based on the models' class accuracy.</font>**

In [ ]:
#Importing all the four different models and store them in variables. The path in the following code need to changed to your actual correct path.
from keras.models import load_model
mk54 = load_model('/content/drive/MyDrive/AI/chest_xray/Giovanna_oversampling_callbacks.h5')
bagley = load_model('/content/drive/MyDrive/AI/MMC/bagle 6.h5')
lola = load_model('/content/drive/MyDrive/AI/MMC/lola.h5')
giovanna = load_model('/content/drive/MyDrive/AI/chest_xray/Giovanna_weight_callbacks.h5')

In [ ]:
#calculating the predicyion of each model
predictions_MK54 = mk54.predict(X_test)
predictions_Bagley = bagley.predict(X_test)
predictions_Giovanna = giovanna.predict(X_test)
predictions_Lola = lola.predict(X_test)

In [ ]:
# Assuming y_val contains the true labels for the validation set
d_fra = {}


final_predictions = []
final_predictions_fra = []
c = 0
for i in range(len(predictions_MK54)):
  final_predictions.append((predictions_Bagley[i][0] + predictions_MK54[i][0] + predictions_Giovanna[i][0] + predictions_Lola[i][0])/ 4)


for i in range(len(predictions_MK54)):
  l = [0] * 4
  l[0] = predictions_Bagley[i][0]
  l[1] = predictions_MK54[i][0]
  l[2] = predictions_Giovanna[i][0]
  l[3] = predictions_Lola[i][0]

  d_fra[i] = l
  c = 0
  for v in range(len(l)):
    if l[v] > 0.5:
      c += 1
    else:
      c -= 1
  if c > 0:
    final_predictions_fra.append(max(l))
  elif c == 0:
    if l[0] > 0.5 or l[1] > 0.5 and l[2] < 0.5 or l[3] < 0.5:
      final_predictions_fra.append(max(l))
    elif l[2] > 0.5 or l[3] > 0.5 and l[0] < 0.5 or l[1] < 0.5:
      final_predictions_fra.append(min(l))
  else:
    final_predictions_fra.append(min(l))



final_predictions_array = np.array(final_predictions)
final_predictions_combined_models_array = np.array(final_predictions_fra)

# print(final_predictions)


# Set a threshold for binary classification
threshold = 0.5

# Round the predicted probabilities to binary predictions
predictions_binary_Bagley = (predictions_Bagley > threshold).astype(int)
predictions_binary_MK54 = (predictions_MK54 > threshold).astype(int)
predictions_binary_Giovanna = (predictions_Giovanna > threshold).astype(int)
predictions_binary_Lola = (predictions_Lola > threshold).astype(int)
predictions_binary_mean = (final_predictions_array > threshold).astype(int)
predictions_binary_combined_models = (final_predictions_combined_models_array > threshold).astype(int)







# Create a DataFrame for better visualization (optional, requires pandas)
import pandas as pd

df_Bagley = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_Bagley.flatten(),
    'Predicted Probability': predictions_Bagley.flatten()
})

df_MK54 = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_MK54.flatten(),
    'Predicted Probability': predictions_MK54.flatten()
})

df_Giovanna = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_Giovanna.flatten(),
    'Predicted Probability': predictions_Giovanna.flatten()
})

df_Lola = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_Lola.flatten(),
    'Predicted Probability': predictions_Lola.flatten()
})

df_mean = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_mean.flatten(),
    'Predicted Probability': final_predictions_array.flatten()
})


df_combined_models = pd.DataFrame({
    'True Label': y_test.flatten(),  # Flatten if y_val is not a 1D array
    'Predicted Binary': predictions_binary_combined_models.flatten(),
    'Predicted Probability': final_predictions_combined_models_array.flatten()
})
# Display the DataFrame
# print(df)
df_Bagley['Predicted Probability (%)'] = (df_Bagley['Predicted Probability']).round(2)
df_MK54['Predicted Probability (%)'] = (df_MK54['Predicted Probability']).round(2)
df_Giovanna['Predicted Probability (%)'] = (df_Giovanna['Predicted Probability']).round(2)
df_Lola['Predicted Probability (%)'] = (df_Lola['Predicted Probability']).round(2)

df_mean['Predicted Probability (%)'] = (df_mean['Predicted Probability']).round(2)
df_combined_models['Predicted Probability (%)'] = (df_combined_models['Predicted Probability']).round(2)

df_Bagley = df_Bagley.drop(columns=['Predicted Probability'])
df_MK54 = df_MK54.drop(columns=['Predicted Probability'])
df_Giovanna = df_Giovanna.drop(columns=['Predicted Probability'])
df_Lola = df_Lola.drop(columns=['Predicted Probability'])

df_mean = df_mean.drop(columns=['Predicted Probability'])
df_combined_models = df_combined_models.drop(columns=['Predicted Probability'])
# print(df.sample(15))

# Print only the rows where predictions are incorrect
incorrect_predictions_Bagley = df_Bagley[df_Bagley['True Label'] != df_Bagley['Predicted Binary']]
incorrect_predictions_MK54 = df_MK54[df_MK54['True Label'] != df_MK54['Predicted Binary']]
incorrect_predictions_Giovanna = df_Giovanna[df_Giovanna['True Label'] != df_Giovanna['Predicted Binary']]
incorrect_predictions_Lola = df_Lola[df_Lola['True Label'] != df_Lola['Predicted Binary']]

incorrect_predictions_mean = df_mean[df_mean['True Label'] != df_mean['Predicted Binary']]
incorrect_predictions_combined_models = df_combined_models[df_combined_models['True Label'] != df_combined_models['Predicted Binary']]


print("Mistakes in prediction Bagley")
print(len(incorrect_predictions_Bagley))

print("Mistakes in prediction MK54")
print(len(incorrect_predictions_MK54))

print("Mistakes in prediction Giovanna")
print(len(incorrect_predictions_Giovanna))

print("Mistakes in prediction Lola")
print(len(incorrect_predictions_Lola))

print("Mistakes in mean prediction")
print(len(incorrect_predictions_mean))

print("Mistakes in fra prediction")
print(len(incorrect_predictions_combined_models))

print("\nIncorrect Predictions Bagley")
print(incorrect_predictions_Bagley)

print("\nIncorrect Predictions MK54")
print(incorrect_predictions_MK54)

print("\nIncorrect Predictions Giovanna")
print(incorrect_predictions_Giovanna)

print("\nIncorrect Predictions Lola:")
print(incorrect_predictions_Lola)

print("\nIncorrect Predictions mean:")
print(incorrect_predictions_mean)

print("\nIncorrect Predictions combined_models:")
print(incorrect_predictions_combined_models)